In [ ]:
import requests
import hashlib
import time
import datetime
from google.colab import userdata
import pandas as pd
import sqlite3

In [ ]:
public_key=userdata.get('public_key_marvel')
private_key=userdata.get('private_key_marvel')
timestamp = str(time.time())
md5_hash= timestamp + private_key + public_key
md5_hash = hashlib.md5(md5_hash.encode("utf-8")).hexdigest()

In [ ]:
# Modifique a URL para o endpoint de personagens
url = f"https://gateway.marvel.com/v1/public/characters/1011334/comics?ts={timestamp}&apikey={public_key}&hash={md5_hash}"

# Código de tratamento dos dados
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(f"Total de personagens: {data['data']['total']}")
    for char in data['data']['results']:
        print(char)

Total de personagens: 12
{'id': 22506, 'digitalId': 10949, 'title': 'Avengers: The Initiative (2007) #19', 'issueNumber': 19, 'variantDescription': '', 'description': '', 'modified': '2017-08-16T12:11:05+0000', 'isbn': '', 'upc': '5960606084-01911', 'diamondCode': 'SEP082362', 'ean': '', 'issn': '', 'format': 'Comic', 'pageCount': 32, 'textObjects': [{'type': 'issue_solicit_text', 'language': 'en-us', 'text': 'SECRET INVASION Tie-In!\r<br>"V-S DAY"\r<br>It\'s been leading to this since the Hank Pym Skrull first came up with the idea for a Fifty State Initiative.  This is the final assault in the Secret Invasion, a nation-wide plan that will test the limits of 3-D MAN\'S superhuman militia, THE KILL KREW ARMY! Join 3-D MAN, CLOUD 9, KOMODO, HARDBALL, and heroes around America in the battle that will decide the fate of the planet and the future of the Initiative program.  Win or lose, there\'s no turning back.  After today, everything changes.\r<br>Rated T+ ...$2.99\r<br>'}, {'type': 'is

# Criação do banco de dados para os characters

In [ ]:
DB_NAME = 'marvel_characters_teste.db'
MAX_RETRIES = 3  # Tentativas por requisição
DELAY = 1  # Atraso entre requisições em segundos

In [ ]:
 # Função para pegar todos os personagens
def get_all_characters():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     offset = 0
     total = None
     inserted = 0

     while True:
         for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 100,
                     'offset': offset
                 }

                 response = requests.get(
                     'https://gateway.marvel.com/v1/public/characters',
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()

                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO characters
                         VALUES (?, ?, ?, ?, ?, ?)
                     ''', (
                         char['id'],
                         char['name'],
                         char.get('description', ''),
                         char['modified'],
                         char['comics']['available'],
                         char['comics']['collectionURI'],
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os personagens foram armazenados!")
                     return

                 offset += 100
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
         else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close()

In [ ]:
 # Função para criar o banco
def create_database():
     conn = sqlite3.connect(DB_NAME)
     cursor = conn.cursor()

     cursor.execute('''
     CREATE TABLE IF NOT EXISTS characters (
         id INTEGER PRIMARY KEY,
         name TEXT NOT NULL,
         description TEXT,
         modified TEXT,
         comics_available INTEGER,
         comics_collectionURI CHAR

     )''')

     cursor.execute('''
     CREATE TABLE IF NOT EXISTS characters_comics (
         id_character INTEGER,
         id_comics INTEGER,
         PRIMARY KEY(id_character, id_comics)
         FOREIGN KEY(id_character) REFERENCES character (id)
     )''')

     conn.commit()
     conn.close()

In [ ]:
create_database()
get_all_characters()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters")
print(f"\nTotal no banco: {cursor.fetchone()[0]} personagens")
conn.close()

Progresso: 100/1564 (6.4%)
Progresso: 200/1564 (12.8%)
Progresso: 300/1564 (19.2%)
Progresso: 400/1564 (25.6%)
Progresso: 500/1564 (32.0%)
Progresso: 600/1564 (38.4%)
Progresso: 700/1564 (44.8%)
Progresso: 800/1564 (51.2%)
Progresso: 900/1564 (57.5%)
Progresso: 1000/1564 (63.9%)
Progresso: 1100/1564 (70.3%)
Progresso: 1200/1564 (76.7%)
Progresso: 1300/1564 (83.1%)
Progresso: 1400/1564 (89.5%)
Progresso: 1500/1564 (95.9%)
Progresso: 1564/1564 (100.0%)

Todos os personagens foram armazenados!

Total no banco: 1564 personagens


In [ ]:
#teste tabela relacionamento personagens e comics

#DB_NAME = 'marvel_characters_teste.db'
#MAX_RETRIES = 3  # Tentativas por requisição
#DELAY = 1  # Atraso entre requisições em segundos

'''def character_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute('''
  #  SELECT id, comics_collectionURI
  #  FROM characters
    ''')

    resultados = cursor.fetchall()


    for id_character, comics_collectionURI in resultados:
     offset = 0
     total = None
     inserted = 0

     while True:
        for attempt in range(MAX_RETRIES):
             try:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 100,
                     'offset': offset
                 }

                 response = requests.get(
                     comics_collectionURI,
                     params=params,
                     timeout=10
                 )
                 response.raise_for_status()

                 data = response.json()
                 total = data['data']['total'] if total is None else total
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                     #    INSERT OR REPLACE INTO characters_comics
                     #    VALUES (?, ?)
                     ''', (
                         id_character,
                         char['id'],
                     ))

                 conn.commit()
                 inserted += len(results)
                 print(f"Progresso: {inserted}/{total} ({inserted/total:.1%})")

                 # Verificar se terminou
                 if offset + len(results) >= total:
                     print("\nTodos os personagens foram armazenados!")
                     return

                 offset += 100
                 time.sleep(DELAY)
                 break  # Sai do loop de tentativas

             except requests.exceptions.RequestException as e:
                 print(f"Erro na requisição (tentativa {attempt+1}): {str(e)}")
                 time.sleep(2 ** attempt)  # Backoff exponencial
                 continue
        else:
             print("Falha após múltiplas tentativas. Abortando...")
             break

     conn.close() '''

In [ ]:
#teste tabela relacionamento personagens e comics 2

DB_NAME = 'marvel_characters_teste.db'

def character_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute('''
    SELECT id, comics_collectionURI
    FROM characters
    ''')

    resultados = cursor.fetchall()

    sum=0
    for id_character, comics_collectionURI in resultados:
      sum+=1
      print(sum,id_character, comics_collectionURI)

character_comics()

1 1009144 https://gateway.marvel.com/v1/public/characters/1009144/comics
2 1009146 https://gateway.marvel.com/v1/public/characters/1009146/comics
3 1009148 https://gateway.marvel.com/v1/public/characters/1009148/comics
4 1009149 https://gateway.marvel.com/v1/public/characters/1009149/comics
5 1009150 https://gateway.marvel.com/v1/public/characters/1009150/comics
6 1009151 https://gateway.marvel.com/v1/public/characters/1009151/comics
7 1009152 https://gateway.marvel.com/v1/public/characters/1009152/comics
8 1009153 https://gateway.marvel.com/v1/public/characters/1009153/comics
9 1009154 https://gateway.marvel.com/v1/public/characters/1009154/comics
10 1009155 https://gateway.marvel.com/v1/public/characters/1009155/comics
11 1009156 https://gateway.marvel.com/v1/public/characters/1009156/comics
12 1009157 https://gateway.marvel.com/v1/public/characters/1009157/comics
13 1009158 https://gateway.marvel.com/v1/public/characters/1009158/comics
14 1009159 https://gateway.marvel.com/v1/public

In [ ]:
#teste tabela relacionamento personagens e comics 2

DB_NAME = 'marvel_characters_teste.db'

def character_comics():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    cursor.execute('''
    SELECT id, comics_collectionURI
    FROM characters
    LIMIT 10
    ''')

    resultados = cursor.fetchall()


    for id_character, comics_collectionURI in resultados:
                 # Autenticação
                 ts = str(time.time())
                 hash_input = ts + private_key + public_key
                 md5_hash = hashlib.md5(hash_input.encode()).hexdigest()

                 params = {
                     'ts': ts,
                     'apikey': public_key,
                     'hash': md5_hash,
                     'limit': 100,
                 }

                 response = requests.get(
                     comics_collectionURI,
                     params=params,
                     timeout=10
                 )

                 data = response.json()
                 results = data['data']['results']

                 # Inserir no banco
                 for char in results:
                     cursor.execute('''
                         INSERT OR REPLACE INTO characters_comics
                         VALUES (?, ?)
                     ''', (
                         id_character,
                         char['id'],
                     ))

                 conn.commit()
    conn.close()

In [ ]:
character_comics()

 # Verificar
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM characters_comics")
print(f"\nTotal no banco: {cursor.fetchone()[0]} linhas")
conn.close()


Total no banco: 496 linhas


# Mostrando o banco marvel_full

In [ ]:
def show_table_sample():
    conn = sqlite3.connect('marvel_characters_teste.db')
    df = pd.read_sql("SELECT * FROM characters_comics", conn)
    conn.close()

    print("\nAmostra dos dados:")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    return df

show_table_sample()


Amostra dos dados:
Dimensões: 496 linhas x 2 colunas


,id_character,id_comics
0,1009144,103120
1,1009144,103371
2,1009144,92308
3,1009144,92307
4,1009144,65941
...,...,...
491,1009155,23328
492,1009155,23327
493,1009155,23326
494,1009155,23325


In [ ]:

conn = sqlite3.connect('marvel_characters.db')
cursor = conn.cursor()

# Executar consulta com parâmetro
cursor.execute('''
    SELECT name, comics_available,series_available,	stories_available,	events_available
    FROM characters
    ORDER BY comics_available DESC
    LIMIT 10
''')

# Obter e mostrar resultados
resultados = cursor.fetchall()

In [ ]:
for name, comics_available,series_available,	stories_available,	events_available	 in resultados:
  print(f"- {name}: comics_available: {comics_available} - series_available: {series_available} - stories_available: {stories_available} - events_available: {events_available}")